## 219. 存在重复元素 II
给定一个整数数组和一个整数 k，判断数组中是否存在两个不同的索引 i 和 j，使得 nums [i] = nums [j]，并且 i 和 j 的差的 绝对值 至多为 k。

示例 1:
```
输入: nums = [1,2,3,1], k = 3
输出: true
```
示例 2:
```
输入: nums = [1,0,1,1], k = 1
输出: true
```
示例 3:
```
输入: nums = [1,2,3,1,2,3], k = 2
输出: false
````

### 标签：哈希
维护一个哈希表，里面始终最多包含 k 个元素，当出现重复值时则说明在 k 距离内存在重复元素

每次遍历一个元素则将其加入哈希表中，如果哈希表的大小大于 k，则移除最前面的数字

时间复杂度：$O(n)$，$n$ 为数组长度

空间复杂度：$O(\min(n, k))$

开辟的额外空间取决于散列表中存储的元素的个数，也就是滑动窗口的大小 $O(\min(n,k))$。



In [36]:
from typing import List

class Solution:
    def containsNearbyDuplicate(self, nums: List[int], k: int) -> bool:
        hash_num = {}
        for i in range(len(nums)):
            if len(list(hash_num.keys())) > k:
                hash_num.pop(list(hash_num.keys())[0])
            if nums[i] in list(hash_num.values()):
                return True
            hash_num[i] = nums[i]
        return False
                    

In [38]:
from typing import List

class Solution:
    def containsNearbyDuplicate(self, nums: List[int], k: int) -> bool:
        arr = {}
        for i in range(len(nums)):
            if nums[i] in arr and i - arr[nums[i]] <= k: return True
            arr[nums[i]] = i
        return False

In [39]:
ss = Solution()
res = ss.containsNearbyDuplicate(nums=[1,0,1,1], k=1)
print(res)

True


## 220. 存在重复元素 III

给你一个整数数组 `nums` 和两个整数 `k` 和 `t` 。请你判断是否存在 两个不同下标 `i` 和 `j`，使得 `abs(nums[i] - nums[j]) <= t` ，同时又满足 `abs(i - j) <= k` 

如果存在则返回 true，不存在返回 false。

 

示例 1：
```
输入：nums = [1,2,3,1], k = 3, t = 0
输出：true
```
示例 2：
```
输入：nums = [1,0,1,1], k = 1, t = 2
输出：true
```
示例 3：
```
输入：nums = [1,5,9,1,5,9], k = 2, t = 3
输出：false
``` 

提示：
```
0 <= nums.length <= 2 * 104
-231 <= nums[i] <= 231 - 1
0 <= k <= 104
0 <= t <= 231 - 1
```

### 滑动窗口 & 二分
根据题意，对于任意一个位置 i（假设其值为 u），我们其实是希望在下标范围为 `[max(0, i - k)` 内找到值范围在 `[u - t, u + t]` 的数。

朴素的想法: 每次遍历到任意位置 i 的时候，往后检查 k 个元素，但复杂度是 $O(nk)$ 的，会超时。

显然我们需要优化「检查后面 k 个元素」这一过程。

我们希望使用一个「有序集合」去维护长度为 k 的滑动窗口内的数，该数据结构最好支持高效「查询」与「插入/删除」操作：

- 查询：能够在「有序集合」中应用「二分查找」，快速找到「小于等于 uu 的最大值」和「大于等于 u 的最小值」（即「有序集合」中的最接近 u 的数）。
- 插入/删除：在往「有序集合」添加或删除元素时，能够在低于线性的复杂度内完成（维持有序特性）。

HashMap 无法很好的支持「范围查询」操作:我们需要找的是符合 $abs(nums[i], nums[j]) \leqslant tabs(nums[i],nums[j])⩽t$ 的两个值，`nums[i]` 与 `nums[j]` 并不一定相等，而 

#### 『树』结构。

例如 AVL，能够让我们在最坏为 $O(\log{k})$ 的复杂度内取得到最接近 u 的值是多少，但本题除了「查询」以外，还涉及频繁的「插入/删除」操作（**随着我们遍历 nums 的元素，滑动窗口不断右移，我们需要不断的往「有序集合」中删除和添加元素**）。

简单采用 AVL 树，会导致每次的插入删除操作都触发 AVL 的平衡调整，一次平衡调整会伴随着若干次的旋转。

而红黑树则很好解决了上述问题：将平衡调整引发的旋转的次数从「若干次」限制到「最多三次」。

因此，当「查询」动作和「插入/删除」动作频率相当时，更好的选择是使用「红黑树」。


In [43]:
from sortedcontainers import SortedList
from typing import List
import bisect

class Solution:
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        window = SortedList()
        for i in range(len(nums)):
            # len(window) == k
            if i > k:
                window.remove[i - k - 1]
            window.add(nums[i])
            idx = bisect.bisect_left(window, nums[i])
            if idx > 0 and abs(window[idx] - window[idx-1]) <= t:
                return True
            if idx < len(window) - 1 and abs(window[idx+1] - window[i]) <= t:
                return True
        return False

In [44]:
ss = Solution()
res = ss.containsNearbyAlmostDuplicate(nums=[1,2,3,1], k = 3, t = 0)
print(res)

False


- 时间复杂度：TreeSet 基于红黑树，查找和插入都是 $O(\log{k})$ 复杂度。整体复杂度为 $O(n\log{k})$
- 空间复杂度：$O(k)$


## 80. 删除有序数组中的重复项 II
给你一个有序数组 nums ，请你 原地 删除重复出现的元素，使每个元素 最多出现两次 ，返回删除后数组的新长度。

不要使用额外的数组空间，你必须在 原地 修改输入数组 并在使用 O(1) 额外空间的条件下完成。


说明：

为什么返回数值是整数，但输出的答案是数组呢？

请注意，输入数组是以「**引用**」方式传递的，这意味着在函数里修改输入数组对于调用者是可见的。

你可以想象内部操作如下:
```
// nums 是以“引用”方式传递的。也就是说，不对实参做任何拷贝
int len = removeDuplicates(nums);

// 在函数里修改输入数组对于调用者是可见的。
// 根据你的函数返回的长度, 它会打印出数组中 该长度范围内 的所有元素。
for (int i = 0; i < len; i++) {
    print(nums[i]);
}
```

示例 1：
```
输入：nums = [1,1,1,2,2,3]
输出：5, nums = [1,1,2,2,3]
解释：函数应返回新长度 length = 5, 并且原数组的前五个元素被修改为 1, 1, 2, 2, 3 。 不需要考虑数组中超出新长度后面的元素。
```
示例 2：
```
输入：nums = [0,0,1,1,1,1,2,3,3]
输出：7, nums = [0,0,1,1,2,3,3]
解释：函数应返回新长度 length = 7, 并且原数组的前五个元素被修改为 0, 0, 1, 1, 2, 3, 3 。 不需要考虑数组中超出新长度后面的元素。
```

提示：
```
1 <= nums.length <= 3 * 104
-104 <= nums[i] <= 104
nums 已按升序排列
```

In [2]:
from typing import List

class Solution:
    def removeDuplicates(self, nums: List[int]) -> int:
        p1,p2 = 0,0
        length = len(nums)
        while 1:
            while nums[p2+1] == nums[p1] and p2+1 < length:
                p2 += 1
            if p2 - p1 > 1:
                p1 += 1 # 移动p1指向第二个相同的数字
                nums = nums[:p1+1] + nums[p2+1:] # delete the extra number
                length -= (p2-p1)  # record the length of nums，length = len(nums) the same
                if nums[p1] != nums[-1]: # if it's the last number, p1 don't need to move
                    p1 += 1 # move the point p1 to the next type of number
                p2 = p1 # move point p2 to point p1
            elif p2 - p1 == 1 or p2 == p1: # if there're not more than 2 same numbers
                if nums[p1] != nums[-1]: # if it's the last number, p1 don't need to move
                    p2 += 1
                p1 = p2
            if nums[p1] == nums[-1]:
                break
                
        return length, nums
            

In [3]:
ss = Solution()
nums = [1,1,1,2,2,3]
res, res_num = ss.removeDuplicates(nums)
nums2 = [0,0,1,1,2,3,3]
res2, res_num_2 = ss.removeDuplicates(nums2)
print(res, res_num)
print(res2, res_num_2)

5 [1, 1, 2, 2, 3]
7 [0, 0, 1, 1, 2, 3, 3]


### 通用解法
将原问题的「保留 2 位」修改为「保留 k 位」

- 由于是保留 k 个相同数字，对于前 k 个数字，我们可以直接保留
- 对于后面的任意数字，能够保留的前提是：与当前写入的位置前面的第 k 个元素进行比较，不相同则保留

我们令 k=2，假设有如下样例
```
[1,1,1,1,1,1,2,2,2,2,2,2,3]
```
- 首先我们先让前 2 位直接保留，得到 1,1
- 对后面的每一位进行继续遍历，能够保留的前提是与当前位置的前面 k 个元素不同（答案中的第一个 1），因此我们会跳过剩余的 1，将第一个 2 追加，得到 1,1,2
- 继续这个过程，这时候是和答案中的第 2 个 1 进行对比，因此可以得到 1,1,2,2
- 这时候和答案中的第 1 个 2 比较，只有与其不同的元素能追加到答案，因此剩余的 2 被跳过，3 被追加到答案：1,1,2,2,3


In [7]:
from typing import List

class Solution:
    def removeDuplicates(self, nums: List[int]) -> int:
        def solve(k):
            u = 0
            for x in nums:
                if u < k or nums[u-k] != x:
                    nums[u] = x
                    u += 1
            return u
        return solve(2)

In [8]:
ss = Solution()
nums = [1,1,1,2,2,3]
res = ss.removeDuplicates(nums)
nums2 = [0,0,1,1,2,3,3]
res2 = ss.removeDuplicates(nums2)
print(res)
print(res2)

5
7


## 27. 移除元素
- 给你一个数组 nums 和一个值 val，你需要 原地 移除所有数值等于 val 的元素，并返回移除后数组的新长度。
- 不要使用额外的数组空间，你必须仅使用 $O(1)$ 额外空间并 原地 修改输入数组。
- 元素的顺序可以改变。你不需要考虑数组中超出新长度后面的元素。

 

说明:
- 为什么返回数值是整数，但输出的答案是数组呢?
- 请注意，输入数组是以「引用」方式传递的，这意味着在函数里修改输入数组对于调用者是可见的。

你可以想象内部操作如下:
```
// nums 是以“引用”方式传递的。也就是说，不对实参作任何拷贝
int len = removeElement(nums, val);

// 在函数里修改输入数组对于调用者是可见的。
// 根据你的函数返回的长度, 它会打印出数组中 该长度范围内 的所有元素。
for (int i = 0; i < len; i++) {
    print(nums[i]);
}
``` 

示例 1：
```
输入：nums = [3,2,2,3], val = 3
输出：2, nums = [2,2]
解释：函数应该返回新的长度 2, 并且 nums 中的前两个元素均为 2。你不需要考虑数组中超出新长度后面的元素。例如，函数返回的新长度为 2 ，而 nums = [2,2,3,3] 或 nums = [2,2,0,0]，也会被视作正确答案。
```
示例 2：
```
输入：nums = [0,1,2,2,3,0,4,2], val = 2
输出：5, nums = [0,1,4,0,3]
解释：函数应该返回新的长度 5, 并且 nums 中的前五个元素为 0, 1, 3, 0, 4。注意这五个元素可为任意顺序。你不需要考虑数组中超出新长度后面的元素。
 ```

提示：
```
0 <= nums.length <= 100
0 <= nums[i] <= 50
0 <= val <= 100
```

In [10]:
from typing import List

class Solution:
    def removeElement(self, nums: List[int], val: int) -> int:
        u = 0
        for x in nums:
            if x != val:
                nums[u] = x
                u += 1
        return u
            

In [12]:
ss = Solution()
res = ss.removeElement(nums = [3,2,2,3], val = 3)
print(res)

2
